In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', delimiter=',', low_memory=False)
submit_data = pd.read_csv('test.csv', delimiter=',', low_memory=False)

In [3]:
#冒頭を表示
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [4]:
#データの形を確認
train.shape

(200000, 77)

In [5]:
#データのタイプを確認
train.dtypes

id             int64
feature_0      int64
feature_1      int64
feature_2      int64
feature_3      int64
               ...  
feature_71     int64
feature_72     int64
feature_73     int64
feature_74     int64
target        object
Length: 77, dtype: object

In [6]:
#trainの基本的統計量を表示
train.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,200000,NaN,NaN,NaN,99999.5,57735.2,0,49999.8,99999.5,149999,199999
feature_0,200000,NaN,NaN,NaN,0.97271,3.94184,0,0,0,1,61
feature_1,200000,NaN,NaN,NaN,1.16837,3.99341,0,0,0,1,51
feature_2,200000,NaN,NaN,NaN,2.21932,6.47657,0,0,0,1,64
feature_3,200000,NaN,NaN,NaN,2.29673,7.55186,0,0,0,1,70
...,...,...,...,...,...,...,...,...,...,...,...
feature_71,200000,NaN,NaN,NaN,0.806895,2.45874,0,0,0,1,30
feature_72,200000,NaN,NaN,NaN,1.28293,4.26142,0,0,0,1,61
feature_73,200000,NaN,NaN,NaN,2.94021,10.7847,0,0,0,1,130
feature_74,200000,NaN,NaN,NaN,0.632005,3.92531,0,0,0,0,52


In [7]:
#trainのtargetをカテゴリーに変換
train.target = train.target.astype('category')

train.dtypes

id               int64
feature_0        int64
feature_1        int64
feature_2        int64
feature_3        int64
                ...   
feature_71       int64
feature_72       int64
feature_73       int64
feature_74       int64
target        category
Length: 77, dtype: object

In [8]:
# ラベルエンコーディング（LabelEncoder）
le = LabelEncoder()
encoded = le.fit_transform(train.target.values)
decoded = le.inverse_transform(encoded)
train.target = encoded

#冒頭を表示して確認
train.target.head()

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64

In [9]:
# 遺伝子座に基づいて特徴量抽出する
drop_count = 0
dna = np.array([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0])
input_num = np.sum(dna)
for i, d in enumerate(dna):
    if d == 0:
        train = train.drop(train.columns[[i+1-drop_count]], axis=1)
        drop_count += 1

# 冒頭を表示して確認
train.head()

,id,feature_0,feature_2,feature_4,feature_5,feature_6,feature_7,feature_8,feature_10,feature_12,...,feature_62,feature_63,feature_64,feature_66,feature_67,feature_68,feature_69,feature_70,feature_72,target
0,0,0,6,0,0,0,0,7,0,3,...,1,0,0,0,0,0,0,0,2,5
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,2,0,0,0,0,0,5
2,2,0,0,0,1,0,3,0,1,0,...,2,0,0,0,0,0,0,1,0,1
3,3,0,7,1,5,2,2,0,2,5,...,7,0,0,0,4,0,2,2,4,7
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
#訓練データを分割する
train_x, train_y = train.drop(['target'], axis=1).drop(['id'], axis=1), train.target

In [11]:
#trainデータを学習データと検証データに分割する
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

#冒頭を表示して確認
x_train.head()

,feature_0,feature_2,feature_4,feature_5,feature_6,feature_7,feature_8,feature_10,feature_12,feature_13,...,feature_58,feature_62,feature_63,feature_64,feature_66,feature_67,feature_68,feature_69,feature_70,feature_72
90017,0,0,0,0,0,0,1,1,2,0,...,0,0,0,0,0,0,0,0,1,0
81636,1,2,2,2,2,0,0,0,7,0,...,4,2,0,1,0,1,0,0,4,0
109466,0,0,0,1,0,0,5,0,8,0,...,0,0,0,0,0,0,0,0,0,0
189641,2,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
165137,0,0,0,3,0,0,0,0,1,0,...,0,3,0,0,0,2,4,2,0,0


In [12]:
#モデルを構築
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(input_num,)),
    keras.layers.Dense(387, activation='relu'),
    keras.layers.Dropout(0.32),
    keras.layers.Dense(364, activation='relu'),
    keras.layers.Dropout(0.18),
    keras.layers.Dense(9, activation='softmax')
])

In [13]:
#モデルをコンパイルする
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
#学習させる
model.fit(x_train, y_train, batch_size=1460, epochs=283)

Epoch 1/283
110/110 [==============================] - 3s 23ms/step - loss: 2.3161 - accuracy: 0.2588
Epoch 2/283
110/110 [==============================] - 3s 26ms/step - loss: 1.8738 - accuracy: 0.3327
Epoch 3/283
110/110 [==============================] - 2s 19ms/step - loss: 1.8255 - accuracy: 0.3442
Epoch 4/283
110/110 [==============================] - 2s 18ms/step - loss: 1.8071 - accuracy: 0.3465
Epoch 5/283
110/110 [==============================] - 2s 17ms/step - loss: 1.7929 - accuracy: 0.3517
Epoch 6/283
110/110 [==============================] - 2s 17ms/step - loss: 1.7808 - accuracy: 0.3524
Epoch 7/283
110/110 [==============================] - 2s 17ms/step - loss: 1.7786 - accuracy: 0.3524
Epoch 8/283
110/110 [==============================] - 2s 22ms/step - loss: 1.7703 - accuracy: 0.3550
Epoch 9/283
110/110 [==============================] - 2s 20ms/step - loss: 1.7721 - accuracy: 0.3528
Epoch 10/283
110/110 [==============================] - 2s 17ms/step - loss: 1.760

110/110 [==============================] - 2s 18ms/step - loss: 1.6154 - accuracy: 0.4102
Epoch 81/283
110/110 [==============================] - 2s 19ms/step - loss: 1.6098 - accuracy: 0.4122
Epoch 82/283
110/110 [==============================] - 2s 19ms/step - loss: 1.6129 - accuracy: 0.4084
Epoch 83/283
110/110 [==============================] - 2s 20ms/step - loss: 1.6088 - accuracy: 0.4123
Epoch 84/283
110/110 [==============================] - 2s 19ms/step - loss: 1.6137 - accuracy: 0.4091
Epoch 85/283
110/110 [==============================] - 2s 18ms/step - loss: 1.6112 - accuracy: 0.4107
Epoch 86/283
110/110 [==============================] - 2s 17ms/step - loss: 1.6098 - accuracy: 0.4091
Epoch 87/283
110/110 [==============================] - 2s 20ms/step - loss: 1.6075 - accuracy: 0.4118
Epoch 88/283
110/110 [==============================] - 2s 18ms/step - loss: 1.6104 - accuracy: 0.4124
Epoch 89/283
110/110 [==============================] - 2s 16ms/step - loss: 1.6047 - 

110/110 [==============================] - 2s 18ms/step - loss: 1.5696 - accuracy: 0.4267
Epoch 160/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5689 - accuracy: 0.4251
Epoch 161/283
110/110 [==============================] - 2s 18ms/step - loss: 1.5694 - accuracy: 0.4247
Epoch 162/283
110/110 [==============================] - 2s 16ms/step - loss: 1.5719 - accuracy: 0.4250
Epoch 163/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5727 - accuracy: 0.4220
Epoch 164/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5699 - accuracy: 0.4259
Epoch 165/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5673 - accuracy: 0.4236
Epoch 166/283
110/110 [==============================] - 2s 19ms/step - loss: 1.5694 - accuracy: 0.4243
Epoch 167/283
110/110 [==============================] - 2s 18ms/step - loss: 1.5714 - accuracy: 0.4247
Epoch 168/283
110/110 [==============================] - 2s 17ms/step - loss: 

110/110 [==============================] - 2s 17ms/step - loss: 1.5511 - accuracy: 0.4306
Epoch 238/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5523 - accuracy: 0.4293
Epoch 239/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5517 - accuracy: 0.4322
Epoch 240/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5559 - accuracy: 0.4284
Epoch 241/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5532 - accuracy: 0.4330
Epoch 242/283
110/110 [==============================] - 2s 19ms/step - loss: 1.5470 - accuracy: 0.4320
Epoch 243/283
110/110 [==============================] - 2s 18ms/step - loss: 1.5513 - accuracy: 0.4328
Epoch 244/283
110/110 [==============================] - 2s 18ms/step - loss: 1.5476 - accuracy: 0.4338
Epoch 245/283
110/110 [==============================] - 2s 17ms/step - loss: 1.5467 - accuracy: 0.4339
Epoch 246/283
110/110 [==============================] - 2s 18ms/step - loss: 

In [15]:
# 検証データへの精度
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

1250/1250 - 1s - loss: 1.8566 - accuracy: 0.3282

Test accuracy: 0.3282249867916107


In [19]:
#提出データを適用できる形にする
# 遺伝子座に基づいて特徴量抽出する
drop_count = 0
for i, d in enumerate(dna):
    if d == 0:
        submit_data = submit_data.drop(submit_data.columns[[i+1-drop_count]], axis=1)
        drop_count += 1

# 冒頭を表示して確認
submit_data.head()

,id,feature_0,feature_2,feature_4,feature_5,feature_6,feature_7,feature_8,feature_10,feature_12,...,feature_58,feature_62,feature_63,feature_64,feature_66,feature_67,feature_68,feature_69,feature_70,feature_72
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,0,0,0,0,0,0,1,2,...,0,7,0,1,1,3,0,0,0,3
2,200002,0,7,0,0,0,0,6,1,0,...,2,0,0,0,0,0,0,0,3,2
3,200003,0,0,3,1,0,0,0,7,0,...,1,0,0,4,0,0,1,0,0,4
4,200004,0,5,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#提出データを適用できる形にする
x_submit = submit_data.drop(['id'], axis=1)

#冒頭を表示して確認
x_submit.head()

,feature_0,feature_2,feature_4,feature_5,feature_6,feature_7,feature_8,feature_10,feature_12,feature_13,...,feature_58,feature_62,feature_63,feature_64,feature_66,feature_67,feature_68,feature_69,feature_70,feature_72
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,2,2,...,0,7,0,1,1,3,0,0,0,3
2,0,7,0,0,0,0,6,1,0,0,...,2,0,0,0,0,0,0,0,3,2
3,0,0,3,1,0,0,0,7,0,0,...,1,0,0,4,0,0,1,0,0,4
4,0,5,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#提出データを識別する
predictions = model.predict(x_submit)

#データの形を確認
predictions.shape

(100000, 9)

In [22]:
#結果を提出形式に変形
df_predictions = pd.DataFrame(predictions)
df_predictions.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submit_data = pd.concat([submit_data.id,df_predictions],axis=1)
submit_data = submit_data.set_index('id')
submit_data.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
200000,0.086867,0.349243,0.144604,0.028992,0.011927,0.173033,0.025344,0.062861,0.117130
200001,0.029691,0.043544,0.034429,0.027634,0.010282,0.243266,0.082336,0.398771,0.130047
200002,0.011590,0.179997,0.040725,0.003332,0.000941,0.539751,0.063115,0.072061,0.088488
200003,0.011402,0.245297,0.029195,0.024251,0.059654,0.219073,0.044584,0.257629,0.108915
200004,0.031247,0.137974,0.035620,0.009600,0.016679,0.335124,0.067292,0.290975,0.075488


In [23]:
# CSVファイルとして出力
submit_data.to_csv("submission_NN_3.csv")